In [2]:
import pandas as pd
import numpy as np

In [4]:
import matplotlib.pyplot as plt

In [5]:
from datetime import datetime

In [9]:
data = pd.read_csv('/Users/djbetts/Desktop/jf_project/recommender_eda.csv')

In [10]:
df = data.copy()

In [17]:
df.drop(['event_params__activity_total_time'], axis=1, inplace=True)

In [22]:
df.columns

Index(['event_name', 'event_date_partition', 'event_params__video_title',
       'event_params__videotitle', 'geo__country', 'geo__city',
       'user_pseudo_id'],
      dtype='object')

In [27]:
df.geo__country.value_counts()

United States    165
India            163
Egypt             41
Bangladesh        36
Nigeria           36
                ... 
Gabon              1
Belgium            1
Palau              1
Azerbaijan         1
Belarus            1
Name: geo__country, Length: 112, dtype: int64

In [23]:
df.event_params__videotitle.value_counts()

Siapakah Tuhan?                    3
Women Disciples                    2
Sinful Woman Forgiven              1
JESUS                              1
Book of Acts                       1
Breathe                            1
Getting Started Is Easy            1
The Story of Jesus for Children    1
Name: event_params__videotitle, dtype: int64

In [25]:
df.event_params__video_title.value_counts()

JESUS                              558
Life of Jesus (Gospel of John)      25
Baptism of Jesus by John            25
Childhood of Jesus                  20
The Beginning                       17
                                  ... 
Lazarus Rises                        1
Episode 2                            1
Episode 12                           1
Stephen Seized                       1
StoryClubs: Jesus and Zacchaeus      1
Name: event_params__video_title, Length: 150, dtype: int64